In [1]:
import glob

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance

from osgeo import gdal
# gdal.SetConfigOption('GDAL_PDF_LAYERS', 'Map_Frame')


In [11]:
# Coordinates of upper left corner, add -0.25N, 0.25W to get lower right
map_corners = [
    (42.25, -74.75, 'NY_Margaretville_144057_1904_62500_geo.pdf'),
    (42.25, -74.5, 'NY_Phoenicia_148207_1903_62500_geo.pdf'),
    (42.25, -74.25, 'NY_Kaaterskill_140118_1892_62500_geo.pdf'),
    (42.25, -74.0, 'NY_Catskill_140458_1895_62500_geo.pdf'),

    (42.0, -74.75, 'NY_Neversink_144205_1910_62500_geo.pdf'),
    (42.0, -74.5, 'NY_Slide_Mountain_148514_1905_62500_geo.pdf'),
    (42.0, -74.25, 'NY_Rosendale_148355_1903_62500_geo.pdf'),
    (42.0, -74.0, 'NY_Rhinebeck_148295_1898_62500_geo.pdf')
]

In [12]:
def get_corners_from_gdal_ds(gdal_ds):
    ulx, xres, _, lry, _, yres = gdal_ds.GetGeoTransform()
    uly = orig_pdf.RasterYSize * yres + lry
    lrx = orig_pdf.RasterXSize * xres + ulx
    return ulx, uly, lrx, lry

# Get approx aspect ratio - just pick random one from the middle
fn = map_corners[len(map_corners)//2][-1]
orig_pdf = gdal.Open(fn)
ulx, uly, lrx, lry = get_corners_from_gdal_ds(orig_pdf)
ASPECT_RATIO = (ulx - lrx) / (uly - lry)

def warp_pdf(map_info):
    uly, ulx, quadrangle = map_info
    orig_pdf = gdal.Open(quadrangle)
    target_size = 4096 # equidistant
    warped = gdal.Warp(
        'warped.tif', orig_pdf, 
        dstSRS='EPSG:4326', # WGS84 
        resampleAlg='cubic',
        width=target_size, height=target_size,
        # this takes min_x, min_y, max_x, max_y
        outputBounds=(ulx, uly - 0.25, ulx + 0.25, uly)
    );
    warped.FlushCache();
    return warped

def gdal_ds_to_image(gdal_ds):
    a1 = gdal_ds.GetRasterBand(1).ReadAsArray()
    a2 = gdal_ds.GetRasterBand(2).ReadAsArray()
    a3 = gdal_ds.GetRasterBand(3).ReadAsArray()

    arr = np.stack((a1, a2, a3), axis=2)
    return Image.fromarray(arr)

def correct_aspect_ratio(im):
    return im.resize(
        (int(im.width * ASPECT_RATIO), im.height), Image.Resampling.BICUBIC
    )

def make_name_readable(fn):
    return fn.split('_14')[0][3:]
    

def do_all_the_things(map_info):
    im = correct_aspect_ratio(gdal_ds_to_image(warp_pdf(map_info)))
    im.save(f'{make_name_readable(map_info[-1])}.tif')
    return im


Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO


In [13]:
def merge_files(map_corners, crop_pixels=(0, 0, 0, 0)):
    
    row_lats = list(set(m[0] for m in map_corners))
    row_lats.sort()
    column_lons = list(set(m[1] for m in map_corners))
    column_lons.sort()

    # n.b. all input files have been transformed to the same size
    for ir, row in enumerate(row_lats):
        print(row)
        maps = [m for m in map_corners if m[0] == row]
        print(', '.join(make_name_readable(m[-1]) for m in maps))

        for ic, col in enumerate(column_lons):
            m = [m for m in maps if m[1] == col][0][-1]
            if ic == 0:
                im = Image.open(f'{make_name_readable(m)}.tif')
                im = crop_image(im, crop_pixels)
                continue 
            im_new = Image.open(f'{make_name_readable(m)}.tif')
            im_new = crop_image(im_new, crop_pixels)
            im = merge_im_horizontally(im, im_new)
        
        if ir == 0:
            im_all = im
            continue
        im_all = merge_im_vertically(im_all, im)

    return im_all

def crop_image(im, n_pixels):
    box = (n_pixels[0], n_pixels[1], im.width - n_pixels[2], im.height - n_pixels[3])
    return im.crop(box)
            
def merge_im_horizontally(im1, im2):
    w = im1.width + im2.width 
    h = im1.height
    im = Image.new("RGB", (w, h))
    im.paste(im1)
    im.paste(im2, (im1.width, 0))
    return im

def merge_im_vertically(im1, im2):
    w = im1.width
    h = im1.height + im2.height 
    im = Image.new("RGB", (w, h))
    im.paste(im2)
    im.paste(im1, (0, im2.height))
    return im

def increase_contrast(im, contrast_scale=3):
    enhancer = ImageEnhance.Contrast(im)
    return enhancer.enhance(contrast_scale)

In [15]:
for m in map_corners:
    if len(glob.glob(f'{make_name_readable(m[-1])}.tif')) == 0: # skip files that have already been processed
        print(make_name_readable(m[-1]))
        im = do_all_the_things(m)


im = merge_files(map_corners, (2, 1, 1, 2))
im.save('test.tif')
# im3 = increase_contrast(im, 1.5)
# sc = 1
# im2 = im3.resize((int(im.width / sc), int(im.height / sc)), Image.Resampling.BICUBIC)
# im2.save('final_map.tif')

Margaretville


Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO


Phoenicia


Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO


Kaaterskill


...

Rosendale


Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO


Rhinebeck


Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
Warning 1: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO


42.0
Neversink, Slide_Mountain, Rosendale, Rhinebeck
42.25
Margaretville, Phoenicia, Kaaterskill, Catskill


In [6]:
im = merge_files(map_corners, (2, 1, 1, 2))
im.save('test.tif')
im.save('final_map.jpg')

42.0
Neversink_144205_1910_62500_geo.pdf, Slide_Mountain_148514_1905_62500_geo.pdf, Rosendale_148355_1903_62500_geo.pdf, Rhinebeck_148295_1898_62500_geo.pdf
42.25
Margaretville_144057_1904_62500_geo.pdf, Phoenicia_148207_1903_62500_geo.pdf, Kaaterskill_130118_1892_62500_geo.pdf, Catskill_140458_1895_62500_geo.pdf
